In [ ]:
from camera_detect import run_detect, load_graph
import cv2
import io
import numpy as np
import tensorflow as tf
from flask import Flask, render_template, Response
from flask_socketio import SocketIO
from PIL import Image

text = ''
app = Flask(__name__)
socketio = SocketIO(app)

imgByteArr = io.BytesIO()

def gen():
    """Video streaming generator function."""
    while True:
        # yield (b'--frame\r\n'
        #        b'Content-Type: image/jpeg\r\n\r\n' + open('./img/current.jpg', 'rb').read() + b'\r\n')
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + imgByteArr.getvalue() + b'\r\n')

@app.route('/')
def index():
    """Video streaming ."""
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

def run_server():
    socketio.run(app, host='172.16.4.10', port=80)

@socketio.on('connect')
def on_connect():
    socketio.emit('serialData', text)
    print('Client connected!')

@socketio.on('disconnect')
def on_disconnect():
    print('Client disconnected!')

@socketio.on('step')
def on_step():
    global text
    global imgByteArr

    result = run_detect(cap, sess, 5)   # 5 samples

    currentImage = Image.fromarray(result[3])
    imgByteArr = io.BytesIO()
    currentImage.save(imgByteArr, format='JPEG')

    line = "Estimated Distance: " + str(result[0]) + " | X Correction: " + str(result[1]) + " | Z Correction: " + str(result[2])
    text += line  + '\n'
    socketio.emit('serialData', text)
    print('Sent: ' + line)

In [1]:
from motion import *



SerialException: [Errno 2] could not open port /dev/coprocessor: [Errno 2] No such file or directory: '/dev/coprocessor'